In [1]:
from expDJ.nb_4Dj import *

In [2]:
# Get MNIST data

x_train,y_train,x_valid,y_valid = get_data()
type(x_train)

torch.Tensor

In [3]:
#export
# Function to normalize the trainig data and validation data
def normalize_to(train, valid):
    m,s = train.mean(),train.std()
    return normalize(train, m, s), normalize(valid, m, s)

In [4]:
# Normalize the x training and vaiation data
x_train,x_valid = normalize_to(x_train,x_valid)
train_ds,valid_ds = Dataset(x_train, y_train),Dataset(x_valid, y_valid)

In [5]:
x_train.mean(), x_train.std()

(tensor(3.0614e-05), tensor(1.))

In [6]:
# Make the datasets
train_ds,valid_ds = Dataset(x_train, y_train),Dataset(x_valid, y_valid) 
nh,bs = 50,512
c = y_train.max().item()+1
loss_func = F.cross_entropy

# Make samplers

train_Smp, valid_Smp = SamplerDJ(train_ds, bs, shuffle=True), SamplerDJ(valid_ds, bs, shuffle=True)

# Make data loaders

train_dl, valid_dl = DataLoaderDJ(train_ds, train_Smp, bs), DataLoaderDJ(valid_ds, valid_Smp, bs)

# Make databunch
db = DataBunchDJ(train_dl, valid_dl, c = c)
db

In [7]:
#export
# Reform MNIST data
class LambdaDJ(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x): return self.func(x)
    
def flatten(x):      return x.view(x.shape[0], -1)
def resize_MNIST(x): return x.view(-1, 1, 28, 28)
        
    


In [8]:
x_train.shape,resize_MNIST(x_train).shape

(torch.Size([50000, 784]), torch.Size([50000, 1, 28, 28]))

In [9]:
def get_cnn_model(data):
    return nn.Sequential(
        LambdaDJ(resize_MNIST),
        nn.Conv2d( 1, 8, 5, padding=2,stride=2), nn.ReLU(), #14
        nn.Conv2d( 8,16, 3, padding=1,stride=2), nn.ReLU(), # 7
        nn.Conv2d(16,32, 3, padding=1,stride=2), nn.ReLU(), # 4
        nn.Conv2d(32,32, 3, padding=1,stride=2), nn.ReLU(), # 2
        nn.AdaptiveAvgPool2d(1),
        LambdaDJ(flatten),
        nn.Linear(32,data.c)
    )

In [10]:
# Model
model = get_cnn_model(db)

In [11]:
# Callbacks
cbfs = [Recorder, partial(AvgStatsCallback,accuracy)]

In [12]:
opt = optim.SGD(model.parameters(), lr=0.4)
learn = LearnerDJ(db, model, opt, loss_func)
run = RunnerDJ(cb_funcs=cbfs)

recorder
avg_stats


In [15]:

%time run.fit(3, learn)

Epoch: 3
Iter: 1
Iter: 2
Iter: 3
Iter: 4
Iter: 5
Iter: 6
Iter: 7
Iter: 8
Iter: 9
Iter: 10
Iter: 11
Iter: 12
Iter: 13
Iter: 14
Iter: 15
Iter: 16
Iter: 17
Iter: 18
Iter: 19
Iter: 20
Iter: 21
Iter: 22
Iter: 23
Iter: 24
Iter: 25
Iter: 26
Iter: 27
Iter: 28
Iter: 29
Iter: 30
Iter: 31
Iter: 32
Iter: 33
Iter: 34
Iter: 35
Iter: 36
Iter: 37
Iter: 38
Iter: 39
Iter: 40
Iter: 41
Iter: 42
Iter: 43
Iter: 44
Iter: 45
Iter: 46
Iter: 47
Iter: 48
Iter: 49
Iter: 50
Iter: 51
Iter: 52
Iter: 53
Iter: 54
Iter: 55
Iter: 56
Iter: 57
Iter: 58
Iter: 59
Iter: 60
Iter: 61
Iter: 62
Iter: 63
Iter: 64
Iter: 65
Iter: 66
Iter: 67
Iter: 68
Iter: 69
Iter: 70
Iter: 71
Iter: 72
Iter: 73
Iter: 74
Iter: 75
Iter: 76
Iter: 77
Iter: 78
Iter: 79
Iter: 80
Iter: 81
Iter: 82
Iter: 83
Iter: 84
Iter: 85
Iter: 86
Iter: 87
Iter: 88
Iter: 89
Iter: 90
Iter: 91
Iter: 92
Iter: 93
Iter: 94
Iter: 95
Iter: 96
Iter: 97
Iter: 98
train: [0.3669805859375, tensor(0.8905)]
valid: [0.16813060302734376, tensor(0.9516)]
Epoch: 3
Iter: 1
Iter: 2
Iter: 3

In [14]:
run.recorder

ERROR:root:No traceback has been produced, nothing to debug.
